In [25]:
###### packages ######
import pandas as pd

import numpy as np
import openpyxl



In [26]:

######### gui to upload files  ######
import PySimpleGUI as sg

sg.theme('Light Blue 2')

layout = [[sg.Text('Enter 2 files to comare, File 1:old file')],
          [sg.Text('File 1', size=(8, 1)), sg.Input(key='Input_1'), sg.FileBrowse()],
          [sg.Text('File 2', size=(8, 1)), sg.Input(key='Input_2'), sg.FileBrowse()],
          [sg.Submit(), sg.Cancel()]]

window = sg.Window('File Compare', layout)

while True:
    event, values = window.read()
    if event in (sg.WINDOW_CLOSED, "Cancel"):
        break
    elif event == 'Submit':
        file_1, file_2 = values['Input_1'], values['Input_2']
        window.close()
        
        #if '' not in [file_1, file_2]:
            # You may need more checks if files exist, if excel files, ...
            # do Excel job here.
            #pass

In [27]:
# Read the older sheet
df = pd.read_excel(file_1, sheet_name=None)

sheet1 = list(df.keys())[1]
df_original = pd.read_excel(file_1,sheet_name=sheet1)

In [28]:
## 
# Read the latest sheet
df1 = pd.read_excel(file_2, sheet_name=None)

sheet2 = list(df1.keys())[1]
df_modified = pd.read_excel(file_2,sheet_name=sheet2)

In [29]:
######### select the required columns ##########

df_original = df_original.iloc[0:,np.r_[0:3] ]

df_original.columns = df_original.iloc[0]
df_original = df_original[1:]

df_modified = df_modified.iloc[0:,np.r_[0:3] ]

df_modified.columns = df_modified.iloc[0]
df_modified = df_modified[1:]

In [30]:
df_original

,Machine Category,Manufacturer Name,Machine Name
1,NaN,1,NaN
2,NaN,NaN,NaN
3,Counter Staff Operated Deposit and Recycler Ma...,ARCA,CM18\nCM18T\nCM18B\nCM18 EVO\nCM18 EVO T\nOM61
4,Counter Staff Operated Deposit and Recycler Ma...,ARCA,CM18\nCM18T\nCM18B\nCM18 EVO\nCM18 EVO T\nOM61
5,Single Note Cash Accepting Components,Astrosystems Ltd,Global Bill Acceptor ST1+
...,...,...,...
167,Desktop Note Counter,Zzap Ltd.,NC60
168,Desktop Note Counter,Zzap Ltd.,NC70
169,Standalone Note Validator,Zzap Ltd.,D40 and D40+
170,Standalone Note Validator,Zzap Ltd.,D40i & D40i+


In [31]:
######### Remove white space and lower case df_original #############

df_original['Machine Name'] = df_original['Machine Name'].str.replace(" ","")
df_original['Machine Category'] = df_original['Machine Category'].str.lower()
df_original['Manufacturer Name'] = df_original['Manufacturer Name'].str.lower()
df_original['Machine Name'] = df_original['Machine Name'].str.lower()
df_original= df_original.where(pd.notnull(df_original), 0)
####### Convert to dictionary ##########
dictionary = df_original.to_dict(orient="index")


In [32]:
######### Remove white space and lower case df_modified #############

df_modified['Machine Name'] = df_modified['Machine Name'].str.replace(" ","")
df_modified['Machine Category'] = df_modified['Machine Category'].str.lower()
df_modified['Manufacturer Name'] = df_modified['Manufacturer Name'].str.lower()
df_modified['Machine Name'] = df_modified['Machine Name'].str.lower()
df_modified= df_modified.where(pd.notnull(df_modified), 0)
####### Convert to dictionary ##########
dictionary1 = df_modified.to_dict(orient="index")


In [33]:
########## Reorder the dataframe rows #########



def dict_reorder(df1,df,dictionary,dictionary1):
    if df1.shape> df.shape:

        key_list = []
        for key1, value1 in dictionary1.items():
            for key, value in dictionary.items():
                if value1 == value:
                    if key1 != key and key not in key_list:
                        temp = dictionary1[key]
                        dictionary1[key] = dictionary1[key1]
                        dictionary1[key1] = temp
                        key_list.append(key)
                    else:
                        pass
        return pd.DataFrame.from_dict(dictionary1, orient='index')
                
    else:
        key_list = []
        for key, value in dictionary.items():
            for key1, value1 in dictionary1.items():
                if value == value1:
                    if key1 != key and key1 not in key_list:
                        temp = dictionary[key1]
                        dictionary[key1] = dictionary[key]
                        dictionary[key] = temp
                        key_list.append(key1)
                    else:
                        pass
        return pd.DataFrame.from_dict(dictionary, orient='index')
    
updated_df = dict_reorder(df_modified,df_original,dictionary,dictionary1)
updated_df


,Machine Category,Manufacturer Name,Machine Name
1,0,0,0
2,0,0,0
3,counter staff operated deposit and recycler ma...,arca,cm18\ncm18t\ncm18b\ncm18evo\ncm18evot\nom61
4,counter staff operated deposit and recycler ma...,arca,cm18\ncm18t\ncm18b\ncm18evo\ncm18evot\nom61
5,single note cash accepting components,astrosystems ltd,globalbillacceptorst1+
...,...,...,...
173,single note cash acceptor components,innovative technology ltd,nv10usb+
174,desktop fitness sorter,masterworks automodules tech corp ltd (uk dist...,nc-7100
175,customer operated deposit and recyclying atms,ncr financial solutions group ltd,ncrselfserv32withgbnaandgbve-i/gbve-iincrselfs...
176,desktop fitness sorter,masterworks automodules tech corp ltd (uk dist...,nc-8500


In [22]:
if df_modified.shape > df_original.shape:
    df_all = pd.concat([df_original, updated_df], 
                   axis='columns', keys=['Previous', 'Latest'])
    df_final = df_all.swaplevel(axis='columns')[df_original.columns[0:]]
else:
    df_all = pd.concat([updated_df, df_modified], 
                   axis='columns', keys=['Previous', 'Latest'])
    df_final = df_all.swaplevel(axis='columns')[updated_df.columns[0:]]
    
df_final

0                                     Machine Category  \
                                              Previous   
1                                                    0   
2                                                    0   
3    counter staff operated deposit and recycler ma...   
4    counter staff operated deposit and recycler ma...   
5                single note cash accepting components   
..                                                 ...   
173                                                NaN   
174                                                NaN   
175                                                NaN   
176                                                NaN   
177                                                NaN   

0                                                      Manufacturer Name  \
                                                Latest          Previous   
1                                                    0                 0   
2                                                    0                 0   
3    counter staff operated deposit and recycler ma...              arca   
4    counter staff operated deposit and recycler ma...              arca   
5                single note cash accepting components  astrosystems ltd   
..                                                 ...               ...   
173               single note cash acceptor components               NaN   
174                             desktop fitness sorter               NaN   
175     customer operated deposit and recyclying atms                NaN   
176                             desktop fitness sorter               NaN   
177                      customer operated deposit atm               NaN   

0                                                       \
                                                Latest   
1                                                    0   
2                                                    0   
3                                                 arca   
4                                                 arca   
5                                     astrosystems ltd   
..                                                 ...   
173                          innovative technology ltd   
174  masterworks automodules tech corp ltd (uk dist...   
175                  ncr financial solutions group ltd   
176  masterworks automodules tech corp ltd (uk dist...   
177                  ncr financial solutions group ltd   

0                                   Machine Name  \
                                        Previous   
1                                              0   
2                                              0   
3    cm18\ncm18t\ncm18b\ncm18evo\ncm18evot\nom61   
4    cm18\ncm18t\ncm18b\ncm18evo\ncm18evot\nom61   
5                         globalbillacceptorst1+   
..                                           ...   
173                                          NaN   
174                                          NaN   
175                                          NaN   
176                                          NaN   
177                                          NaN   

0                                                       
                                                Latest  
1                                                    0  
2                                                    0  
3          cm18\ncm18t\ncm18b\ncm18evo\ncm18evot\nom61  
4          cm18\ncm18t\ncm18b\ncm18evo\ncm18evot\nom61  
5                               globalbillacceptorst1+  
..                                                 ...  
173                                           nv10usb+  
174                                            nc-7100  
175  ncrselfserv32withgbnaandgbve-i/gbve-iincrselfs...  
176                                            nc-8500  
177  ncrselfserv32withsdm/dvmncrselfserv34withsdm/d...  

[177 rows x 6 columns]

In [34]:
def highlight_diff(data, color='yellow'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('Previous', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

df_coloured = df_final.reset_index(drop=True).style.apply(highlight_diff, axis=None)
df_coloured

In [35]:
df_coloured.to_excel(r"C:\Users\nilanjan.das\Downloads\competitive_test\Result.xlsx", engine = 'openpyxl')

In [93]:
#df_coloured.to_excel(r".\Result.xlsx", engine = 'openpyxl')